___
<h1> Machine Learning </h1>
<h2> Systems Engineering and Computer Technologies / Engenharia de Sistemas e Tecnologias Informáticas
(LESTI)</h2>
<h3> Instituto Superior de Engenharia / Universidade do Algarve </h3>

[LESTI](https://ise.ualg.pt/curso/1941) / [ISE](https://ise.ualg.pt) / [UAlg](https://www.ualg.pt)

Pedro J. S. Cardoso (pcardoso@ualg.pt)

___

# Text Data: Regular Expressions and Basic Pre-processing

In this notebook we'll see how to use regular expressions to extract structured information from text data.

In [1]:
import pandas as pd

df = pd.read_csv('./data/carros-usados/train.csv')

print("Dataframe's shape = ", df.shape)
df.head(20)

Dataframe's shape =  (188533, 13)


,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,0,MINI,Cooper S Base,2007,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,None reported,Yes,4200
1,1,Lincoln,LS V8,2002,143250,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,At least 1 accident or damage reported,Yes,4999
2,2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,None reported,Yes,13900
3,3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,45000
4,4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,None reported,Yes,97500
5,5,Audi,A6 2.0T Sport,2018,40950,Gasoline,252.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,A/T,White,–,None reported,Yes,29950
6,6,Audi,A8 L 3.0T,2016,62200,Gasoline,333.0HP 3.0L V6 Cylinder Engine Gasoline Fuel,8-Speed A/T,Black,Black,None reported,Yes,28500
7,7,Chevrolet,Silverado 1500 1LZ,2016,102604,E85 Flex Fuel,355.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,White,Gray,None reported,Yes,12500
8,8,Ford,F-150 XLT,2020,38352,Gasoline,2.7L V6 24V PDI DOHC Twin Turbo,10-Speed Automatic,Snowflake White Pearl Metallic,Black,None reported,Yes,62890
9,9,BMW,M4 Base,2015,74850,Gasoline,425.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,Transmission w/Dual Shift Mode,Black,Blue,None reported,Yes,4000


## Regular Expressions for Data Extraction

Regular expressions (regex) are powerful tools for extracting structured information from text data. In this notebook, we'll use regex to extract specific features from the `engine` column of our used cars dataset.

The engine column contains complex strings with multiple pieces of information:
- Horsepower (HP)
- Engine displacement (Liters)
- Number of cylinders
- Engine configuration (V6, V8, Straight, etc.)
- Fuel type (Gasoline, Diesel, Electric, Hybrid, etc.)

Let's explore how to extract each of these features systematically.


### 1. Exploring the Engine Column

First, let's examine some sample values from the engine column to understand the different patterns we need to handle.


In [2]:
# Display unique engine patterns (showing first 20)
print("Sample engine descriptions:")
print("-" * 80)
for i, engine in enumerate(df['engine'].dropna().unique()[:20], 1):
    print(f"{i:2d}. {engine}")


Sample engine descriptions:
--------------------------------------------------------------------------------
 1. 172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel
 2. 252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel
 3. 320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capability
 4. 420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel
 5. 208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel
 6. 252.0HP 2.0L 4 Cylinder Engine Gasoline Fuel
 7. 333.0HP 3.0L V6 Cylinder Engine Gasoline Fuel
 8. 355.0HP 5.3L 8 Cylinder Engine Flex Fuel Capability
 9. 2.7L V6 24V PDI DOHC Twin Turbo
10. 425.0HP 3.0L Straight 6 Cylinder Engine Gasoline Fuel
11. 312.0HP 3.6L V6 Cylinder Engine Gasoline Fuel
12. Electric Motor Electric Fuel System
13. 420.0HP 6.2L 8 Cylinder Engine Gasoline Fuel
14. 523.0HP 4.4L 8 Cylinder Engine Gasoline Fuel
15. 503.0HP 4.0L 8 Cylinder Engine Gasoline Fuel
16. 395.0HP 3.0L Straight 6 Cylinder Engine Gasoline/Mild Electric Hybrid
17. 310.0HP 4.6L 8 Cylinder Engine Gasoline Fuel
18. 469.0HP 4.0L 8 Cylinder En

### 2. Extracting Horsepower (HP)

The horsepower is typically indicated by a number followed by "HP" (e.g., "172.0HP", "252HP", "420.0HP").

**Regex pattern explanation:**
- `(\d+\.?\d*)` - Captures one or more digits, optionally followed by a decimal point and more digits
- `HP` - Matches the literal text "HP"
- The parentheses `()` create a capturing group to extract just the numeric value


In [3]:
import re

# Extract horsepower using regex
df['horsepower'] = df['engine'].str.extract(r'(\d+\.?\d*)HP', expand=False).astype(float)

# Show some examples
print("Examples of horsepower extraction:")
df[['engine', 'horsepower']].head(10)

Examples of horsepower extraction:


,engine,horsepower
0,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,172.0
1,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,252.0
2,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,320.0
3,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,420.0
4,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,208.0
5,252.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,252.0
6,333.0HP 3.0L V6 Cylinder Engine Gasoline Fuel,333.0
7,355.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,355.0
8,2.7L V6 24V PDI DOHC Twin Turbo,NaN
9,425.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,425.0


In [4]:
# Check for missing values
print(f"\nMissing horsepower values: {df['horsepower'].isna().sum()} out of {len(df)} ({df['horsepower'].isna().sum()/len(df)*100:.2f}%)")



Missing horsepower values: 33259 out of 188533 (17.64%)


### 3. Extracting Engine Displacement (Liters)

Engine displacement is indicated by a number followed by "L" (e.g., "1.6L", "3.9L", "5.3L").

**Regex pattern explanation:**
- `(\d+\.?\d*)` - Captures one or more digits with optional decimal point
- `L` - Matches the literal "L" (liters)
- We use a word boundary `\b` or look ahead to ensure we match "L" as a unit marker, not part of another word


In [5]:
# Extract engine displacement (liters)
df['displacement_L'] = df['engine'].str.extract(r'(\d+\.?\d*)L\s', expand=False).astype(float)

# Show some examples
print("Examples of displacement extraction:")
df[['engine', 'displacement_L']].head(10)

Examples of displacement extraction:


,engine,displacement_L
0,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,1.6
1,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,3.9
2,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,5.3
3,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,5.0
4,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,2.0
5,252.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,2.0
6,333.0HP 3.0L V6 Cylinder Engine Gasoline Fuel,3.0
7,355.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,5.3
8,2.7L V6 24V PDI DOHC Twin Turbo,2.7
9,425.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,3.0


In [6]:
# Statistics
print(f"\nDisplacement statistics:")
print(df['displacement_L'].describe())
print(f"\nMissing displacement values: {df['displacement_L'].isna().sum()} out of {len(df)} ({df['displacement_L'].isna().sum()/len(df)*100:.2f}%)")



Displacement statistics:
count    174244.000000
mean          3.806903
std           1.356152
min           0.650000
25%           3.000000
50%           3.500000
75%           4.800000
max           8.400000
Name: displacement_L, dtype: float64

Missing displacement values: 14289 out of 188533 (7.58%)


### 4. Extracting Number of Cylinders

The number of cylinders appears before the word "Cylinder" (e.g., "4 Cylinder", "8 Cylinder", "6 Cylinder"). 

Some engines also have configurations like "V6", "V8", "V10" where the number indicates cylinders.

**Regex pattern explanation:**
- `(\d+)\s+Cylinder` - Captures digits followed by space(s) and the word "Cylinder"
- We'll use multiple patterns to catch different formats


In [7]:
# Extract number of cylinders
# First try to extract from "X Cylinder" pattern
df['cylinders'] = df['engine'].str.extract(r'(\d+)\s+Cylinder', expand=False)

# For cases where it's not found, try to extract from V6, V8, V10 pattern
df['cylinders'] = df['cylinders'].fillna(df['engine'].str.extract(r'V(\d+)', expand=False))

# Convert to number
df['cylinders'] = df['cylinders'].astype(float)

# Show some examples
print("Examples of cylinder extraction:")
print(df[['engine', 'cylinders']].head(10))

Examples of cylinder extraction:
                                              engine  cylinders
0       172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel        4.0
1       252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel        8.0
2  320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...        8.0
3       420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel        8.0
4       208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel        4.0
5       252.0HP 2.0L 4 Cylinder Engine Gasoline Fuel        4.0
6      333.0HP 3.0L V6 Cylinder Engine Gasoline Fuel        6.0
7  355.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...        8.0
8                    2.7L V6 24V PDI DOHC Twin Turbo        6.0
9  425.0HP 3.0L Straight 6 Cylinder Engine Gasoli...        6.0


In [8]:
# Show distribution
print(f"\nCylinder distribution:")
print(df['cylinders'].value_counts().sort_index())


Cylinder distribution:
cylinders
3.0       147
4.0     29051
5.0       627
6.0     74655
8.0     59532
10.0     1087
12.0     1671
Name: count, dtype: int64


### 8. Extracting Engine Technologies

Modern engines often have advanced technologies indicated in their descriptions:
- **Turbo** (Turbocharged, Twin Turbo)
- **Supercharged**
- **DOHC** (Double Overhead Camshaft)
- **OHV** (Overhead Valve)
- **PDI** (Port Direct Injection)
- **GDI** (Gasoline Direct Injection)
- **MPFI** (Multi-Point Fuel Injection)

**Regex pattern explanation:**
- We'll use multiple patterns to identify these technologies
- `(?i)` makes the pattern case-insensitive


In [9]:
# Extract turbo information
df['is_turbo'] = df['engine'].str.contains(r'(?i)turbo', na=False)

# Extract specific turbo types
df['turbo_type'] = df['engine'].str.extract(r'(Twin Turbo|Turbo)', expand=False)

# Extract valve train technology (DOHC, OHV, etc.)
df['valve_train'] = df['engine'].str.extract(r'(DOHC|OHV|SOHC)', expand=False)

# Extract fuel injection technology
df['fuel_injection'] = df['engine'].str.extract(r'(PDI|GDI|MPFI)', expand=False)

# Show examples
print("Examples of technology extraction:")
df[['engine', 'is_turbo', 'turbo_type', 'valve_train', 'fuel_injection']].head(20)

Examples of technology extraction:


,engine,is_turbo,turbo_type,valve_train,fuel_injection
0,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,False,NaN,NaN,NaN
1,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,False,NaN,NaN,NaN
2,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,False,NaN,NaN,NaN
3,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,False,NaN,NaN,NaN
4,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,False,NaN,NaN,NaN
5,252.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,False,NaN,NaN,NaN
6,333.0HP 3.0L V6 Cylinder Engine Gasoline Fuel,False,NaN,NaN,NaN
7,355.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,False,NaN,NaN,NaN
8,2.7L V6 24V PDI DOHC Twin Turbo,True,Twin Turbo,DOHC,PDI
9,425.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,False,NaN,NaN,NaN


In [10]:
# Statistics
print(f"\nTurbocharged engines: {df['is_turbo'].sum()} ({df['is_turbo'].sum()/len(df)*100:.2f}%)")
print(f"\nTurbo type distribution:")
print(df['turbo_type'].value_counts())
print(f"\nValve train distribution:")
print(df['valve_train'].value_counts())
print(f"\nFuel injection distribution:")
print(df['fuel_injection'].value_counts())



Turbocharged engines: 14270 (7.57%)

Turbo type distribution:
turbo_type
Turbo         7406
Twin Turbo    6864
Name: count, dtype: int64

Valve train distribution:
valve_train
DOHC    19534
OHV      3943
SOHC      607
Name: count, dtype: int64

Fuel injection distribution:
fuel_injection
GDI     14010
MPFI     4582
PDI      4331
Name: count, dtype: int64


### 9. Summary: All Extracted Features

Let's create a summary view of all the features we've extracted from the engine column using regular expressions.


In [11]:
# Display all extracted features together
extracted_features = ['horsepower', 'displacement_L', 'cylinders', 'is_turbo',
'turbo_type', 'valve_train', 'fuel_injection']

print("Sample of extracted features from engine column:")
display_cols = ['engine'] + extracted_features
df[display_cols].head(15)


Sample of extracted features from engine column:


,engine,horsepower,displacement_L,cylinders,is_turbo,turbo_type,valve_train,fuel_injection
0,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,172.0,1.6,4.0,False,NaN,NaN,NaN
1,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,252.0,3.9,8.0,False,NaN,NaN,NaN
2,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,320.0,5.3,8.0,False,NaN,NaN,NaN
3,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,420.0,5.0,8.0,False,NaN,NaN,NaN
4,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,208.0,2.0,4.0,False,NaN,NaN,NaN
5,252.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,252.0,2.0,4.0,False,NaN,NaN,NaN
6,333.0HP 3.0L V6 Cylinder Engine Gasoline Fuel,333.0,3.0,6.0,False,NaN,NaN,NaN
7,355.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,355.0,5.3,8.0,False,NaN,NaN,NaN
8,2.7L V6 24V PDI DOHC Twin Turbo,NaN,2.7,6.0,True,Twin Turbo,DOHC,PDI
9,425.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,425.0,3.0,6.0,False,NaN,NaN,NaN


### 12. Regex Pattern Reference Guide

Here's a comprehensive reference of the regex patterns we used and their explanations:

| Pattern | Description | Example Match |
|---------|-------------|---------------|
| `\d` | Matches any digit (0-9) | "5" in "5.0L" |
| `\d+` | Matches one or more digits | "172" in "172HP" |
| `\d+\.?\d*` | Matches decimal numbers | "3.5" or "300" |
| `\s` | Matches whitespace (space, tab, newline) | Space in "4 Cylinder" |
| `\s+` | Matches one or more whitespace characters | Spaces |
| `()` | Capturing group - extracts matched content | Extracts "252" from "(252)HP" |
| `|` | OR operator - matches either pattern | "Gasoline|Diesel" |
| `?` | Makes preceding element optional | "\.?" makes decimal point optional |
| `*` | Matches 0 or more of preceding element | "\d*" matches any number of digits |
| `+` | Matches 1 or more of preceding element | "\d+" matches at least one digit |
| `(?i)` | Case-insensitive flag | Matches "turbo", "TURBO", "Turbo" |
| `\b` | Word boundary | Ensures match is a complete word |
